In [1]:
import findspark
findspark.init()


In [2]:
import pyspark


from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]")\
                            .appName("SparkStreamingKafkaBasic").getOrCreate()



In [3]:
# read from kafka, here spark is consumer for kafka topic called test
# spark streaming works as dataframe/sql
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "192.168.93.128:9092")\
  .option("subscribe", "invoices")\
  .load()
 

In [4]:
# .show/print will not work directily due to stream..
# linesDf.show() # worn't work
kafkaDf.printSchema() # works

# key is kafka key, in binary format
# value is kafka value, in binary format
# topic string
# parition, integer
# offer long 
# timestamp - longint in ms
# timestampType - Source Time, Record write time

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
# now convert kafka value which is in bytes to STRING, we ignore the key for now...
# now we pick only value from the stream..
linesDf = kafkaDf.selectExpr("CAST(value AS STRING)")
linesDf.printSchema() # we get only value as string

root
 |-- value: string (nullable = true)



In [6]:
# to print the data on console..
# read the data send by nc command from linux terminal, print it on Jupyter console
echoOnconsole = linesDf\
                .writeStream\
                .outputMode("append")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

In [ ]:
echoOnconsole.awaitTermination()

# later you can terminal the jupyter